Implementation of Pseudo-Random Number Generators specifically the Mersenne Twister and Blum Blum Shub algorithms

# **Mersenne Twister**

In [107]:
# Variables to be used in the code
n = 624  
idx = n+1

# Arrays to hold variables for computation
int_ = [32, 101, 55, 11, 7, 17, 22, 17953]
hex_ =  [0xFFFFFFFF, 0x9D2C5680, 0xEFC60000, 0x9908B0DF]
limits_ = [0x7FFFFFFF, 0x80000000] #lower, upper masks

# Generator state array
gen_array = [0 for i in range(n)]


# Set Seed value and initialize Generator array for MT state
def setSeed(seed):
    gen_array[0] = seed
    for i in range(1, n):
        num = int_[7] * (gen_array[i-1] ^ (gen_array[i-1] >> (int_[0]-2))) + i
        gen_array[i] = num & 0xffffffff


# Generate the next n values for the Generator state array
def nextN():

    for i in range(0, n):
        x = (gen_array[(i+1) % n] & limits_[0]) + (gen_array[i] & limits_[1]) 
        x_ = x >> 1
        #modulo check
        if (x % 2) != 0:
            x_ ^= hex_[3]
        gen_array[i] = gen_array[(i + int_[1]) % n] ^ x_


# Generate Random Number using Generator array 
def genRandom():

    global idx
    if idx >= n:
        nextN()
        idx = 0
    y = gen_array[idx]
    y = y ^ ((y >> int_[3]) & hex_[0])
    y = y ^ ((y << int_[4]) & hex_[1])
    y = y ^ ((y << int_[5]) & hex_[2])
    y = y ^ (y >> int_[6])
    idx += 1
    rand = y & hex_[1]

    return rand



if __name__ == '__main__':
    # Set seed value 
    setSeed(0)
    # print random number
    print(genRandom())


203425280


# *Blum Blum Shub (BBS)*

In [108]:
# Inbuilt Python modules to be used
import random
import math


# Calculate GCD
def calcGCD(n1,n2):
    n1,n2=(n2,n1) if n1<n2 else (n1,n2)
    while n2:
        n1,n2=n2,n1%n2

    #return GCD
    return n1



# Generate seed value using Random module
def seedVal():
    seed = random.randint(1, n-1)
    # make the seed odd
    seed = seed | 1
    while calcGCD(n, seed) != 1:
        seed = random.randint(1, n-1)

    #return seed
    return seed



# Check if number (num) is prime.
def checkPrime(num):
    for j in range(100):
      n = random.randint(2,num-1)
      if pow(n, num - 1, num) != 1:
          return 0
    # check if number is congruent to 1 mod 4
    if num % 4 == 1:
        return 0

    # return 1 if num is prime, else 0
    return 1



# Generate p and q of size (size_) bits
def genPrimes(size_):

    # Setting up p
    # padding m
    m = 1 << size_ - 1

    p_num = random.getrandbits(size_)
    p_num = p_num | m

    # make odd
    p_num = p_num | 1

    # If not Prime
    while not checkPrime(p_num):
        p_num = random.getrandbits(size_)
        p_num = p_num | m
        p_num = p_num | 1

    # Setting up q
    q_num = random.getrandbits(size_)
    q_num = q_num | m
    q_num = q_num | 1
    while not checkPrime(q_num):
        q_num = random.getrandbits(size_)
        q_num = q_num | m
        q_num = q_num | 1

    # obtain p and q
    p = p_num 
    q = q_num

    # return p and q
    return p, q


# Generate random numbers with primes x_bits long while the output is l bits long
def genRandomNum(size_):
    # generate primes with size (bits) long
    p = genPrimes(size_)
    # n = p*q
    n = p[0]*p[1]
    seed = seedVal()
    # Random number
    random_num = pow(seed, 2, n)
  
    return random_num



if __name__ == '__main__':
    # print random number 20 bits long
    print(genRandomNum(20))


172225


# **TESTS**

In [109]:
import numpy as np
from math import gamma,e

def upper_incomplete_gamma(a,x,d=0,iterations=100):
    if d == iterations:
        if ((d % 2) == 1):
            return 1.0
        else:
            m = d/2
            return x + (m-a)
    if d == 0:
        try:
            result = ((x**a) * (e**(-x)))/upper_incomplete_gamma(a,x,d=d+1)
        except OverflowError:
            result = 0.0
        return result
    elif ((d % 2) == 1):
        m = 1.0+((d-1.0)/2.0)
        return x+ ((m-a)/(upper_incomplete_gamma(a,x,d=d+1)))
    else:
        m = d/2
        return 1+(m/(upper_incomplete_gamma(a,x,d=d+1)))
    
def upper_incomplete_gamma2(a,x,d=0,iterations=100):
    if d == iterations:
        return 1.0 
    if d == 0:
        result = ((x**a) * (e**(-x)))/upper_incomplete_gamma2(a,x,d=d+1)
        return result
    else:
        m = (d*2)-1
        return (m-a)+x+ ((d*(a-d))/(upper_incomplete_gamma2(a,x,d=d+1)))
    
def lower_incomplete_gamma(a,x,d=0,iterations=100):
    if d == iterations:
        if ((d % 2) == 1):
            return 1.0 
        else:
            m = d/2
            return x + (m-a)
    if d == 0:
        result = ((x**a) * (e**(-x)))/lower_incomplete_gamma(a,x,d=d+1)
        return result
    elif ((d % 2) == 1):
        m = d - 1
        n = (d-1.0)/2.0
        return a + m - (((a+n)*x)/lower_incomplete_gamma(a,x,d=d+1))
    else:
        m = d-1
        n = d/2.0
        return a+m+((n*x)/(lower_incomplete_gamma(a,x,d=d+1)))
    
def lower_incomplete_gamma2(a,x):
    return gamma(a)-upper_incomplete_gamma2(a,x)
def complimentary_incomplete_gamma(a,x):
    return 1.0-upper_incomplete_gamma(a,x)
def gammainc(a,x):
    return lower_incomplete_gamma(a,x)/gamma(a)
def gammaincc(a,x):
    return upper_incomplete_gamma(a,x)/gamma(a)
   


Collecting Random numbers from our algorithms for analysis

In [112]:
# Sample Test Collecting 1000 numbers from PRNGs

randNumberMT = []
randNumbersBSS = []

#collect from BBS
for i in range(1000):
    randNumbersBSS.append(genRandomNum(20))

#collect from MT
for j in range(1000):
    # Set seed value 
    setSeed(0)
    # print random number
    randNumberMT.append(genRandom())


In [ ]:
#choose list to convert to binary for tests. (Either randNumberMT for MT, or randNmberBSS for BSS)
data_list = randNumberMT   

data = np.array(data_list)

bin_data = data.tostring()

data_median = statistics.median(data) 

Tests

In [114]:
# Tests

# Imports
from fractions import Fraction
from math import erfc
import numpy as np
import statistics

#import our local script (tests_setup.py) for tests
#from tests import *

# Test 1: Monobit Test

def monobitTest(data:str):

    count = 0

    for char in data:
        if char == '0':
            count -= 1
        else:
            count += 1

    sobs = count / math.sqrt(len(bin_data))

    p_val = erfc(math.fabs(sobs) / math.sqrt(2))

    print(f'Monobit Test P-Value = {p_val}')

    return p_val




# Test 2: Runs Test

def runsTest(data, data_median): 
  
    runs, x, y = 0, 0, 0
      
    for i in range(len(data)): 
          
        if (data[i] >= data_median and data[i-1] < data_median) or (data[i] < data_median and data[i-1] >= data_median): 
            runs += 1  
          
        if(data[i]) >= data_median: 
            x += 1   
          
        else: 
            y += 1   
  
    runs_exp = ((2*x*y)/(x+y))+1

    stan_dev = np.sqrt((2*x*y*(2*x*y-x-y))/  
                       (((x+y)**2)*(x+y-1))) 
  
    z = (runs-runs_exp)/stan_dev 

    return z




# Test 3: Frequency Test

def frequencyTest(bin_data):

    # Compute number of blocks M = block size. N = num of blocks (N = floor(n/M))
    # miniumum block size 10 bits, most blocks 100
    n = len(bin_data)

    M = 10

    N = int(math.floor(n/M))

    if N > 99:

        N=99

        M = int(math.floor(n/N))

    num_of_blocks = N

    block_size = M 

    proportions = list()

    for i in range(num_of_blocks):

        block = bin_data[i*(block_size):((i+1)*(block_size))]

        zeroes,ones = counts(block)

        proportions.append(Fraction(ones,block_size))

    chisq = 0.0

    for prop in proportions:

        chisq += 4.0*block_size*((prop - Fraction(1,2))**2)
    
    p = gammaincc((num_of_blocks/2.0),float(chisq)/2.0)

    success = (p >= 0.01)

    print (f'Frequency Test P-Value: {p}')

    return 

def counts(data):

    ones = 0

    zeroes = 0

    for bit in data:

        if (bit == 1):

            ones += 1

        else:

            zeroes += 1

    return zeroes,ones


# Output results of tests
if __name__ == '__main__':

    #print(f'Random Numbers Array: {data}')
    print('-----------------------------------------------')
    monobitTest(bin_data)
    print('-----------------------------------------------')
    Z = runsTest(data, data_median)
    print(f'Z-test Stats Value = {Z}')
    print('-----------------------------------------------')
    frequencyTest(data)


-----------------------------------------------
Monobit Test P-Value = 0.0
-----------------------------------------------
Z-test Stats Value = 0.06327723141882918
-----------------------------------------------
Frequency Test P-Value: 6.597254963845854e-147
